Hugging Face Transformers와 datasets 패키지를 바탕으로 트랜스포머를 활용하기 위한 기본 내용 실습. transformers, datasets, pytorch 패키지가 설치되어 있어야 함.

In [ ]:
from google.colab import drive

use_colab = True  # Colab을 사용하는 경우 True, 아니면 False로 설정하기 바람.

# Google Colab 사용하는 경우 구글 드라이브 마운트
if use_colab:
    drive.mount('/content/drive')
    local_path = '/content/drive/MyDrive/tmclass/'
else:
    local_path = './'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 필요한 패키지 설치가 안되어 있거나, Google Colab에서 실행하는 경우 이 셀을 먼저 실행하기 바람.
!pip install transformers datasets torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 데이터셋
`datasets.load_dataset()` 함수를 통해서 데이터셋을 다운로드/로딩할 수 있음.
허깅페이스 허브에 어떤 데이터셋이 있는지는 https://huggingface.co/datasets 에서 검색해 볼 수 있음.
허깅페이스 데이터셋이 아니더라도 원격/로컬에 저장되어 있는 자신의 데이터 파일을 `datasets.load_dataset()`로 로딩할 수 있음.
리턴값은 `Dataset` 또는 `DatasetDict` 객체.

`datasets.load_dataset()` 함수의 주요 파라미터:
* `path`: 로딩 스크립트(\*.py)나 허깅페이스 데이터셋 이름. 로딩 스크립트에 'csv', 'text', 'json' 등 설정 가능. 허깅페이스 허브의 데이터셋 목록은 `datasets.list_datasets()` 함수를 통해 확인 가능.
* `data_files`: 데이터 파일의 경로(들). str, 시퀀스, 매핑 등 가능.
* `data_dir`: 데이터 디렉토리. 이 디렉토리 내 모든 파일 로딩.
* `split`: 어떤 데이터 분할(train/validation/test)을 로딩할 것인지 설정. 이 값이 지정되면 `Dataset` 객체가 리턴되고, 그렇지 않은 경우 `DatasetDict` 객체가 리턴됨. 어떤 분할이 정의되어 있는지는 `datasets.get_dataset_split_names()` 함수로 확인 가능.

`datasets.load_dataset()` 함수 사용 예:
* 허깅페이스 데이터셋 로딩: `datasets.load_dataset('데이터셋 이름')`
* 데이터 파일 로딩(원격 또는 로컬):
  * `datasets.load_dataset('로딩 스크립트', data_files='데이터 파일(들)')`
  * `datasets.load_dataset('로딩 스크립트', data_dir='데이터 디렉토리')`

Pandas DataFrame에서 데이터셋을 로딩하려면 `datasets.Dataset.from_pandas()` 함수 이용.

상세 사항은 허깅페이스 공식 문서 참고: https://huggingface.co/docs/datasets/index

In [ ]:
# 허깅페이스 허브에 있는 데이터셋 목록 확인
from datasets import list_datasets

all_datasets = list_datasets()
print(f"현재 허브에는 {len(all_datasets)}개의 데이터셋이 있습니다.")
print(f"처음 10개 데이터셋: {all_datasets[:10]}")

현재 허브에는 32376개의 데이터셋이 있습니다.
처음 10개 데이터셋: ['acronym_identification', 'ade_corpus_v2', 'adversarial_qa', 'aeslc', 'afrikaans_ner_corpus', 'ag_news', 'ai2_arc', 'air_dialogue', 'ajgt_twitter_ar', 'allegro_reviews']


In [ ]:
# 허깅페이스 허브의 감정 분류 데이터셋 로딩
from datasets import load_dataset
emotions = load_dataset('emotion')

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# train / validation / test 등 3가지 데이터셋으로 구분되어 있음
emotions

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [ ]:
from datasets import get_dataset_split_names
get_dataset_split_names('emotion')

['train', 'validation', 'test']

In [ ]:
# 각 데이터셋은 'text'와 'label'로 구분됨.
emotions_train = emotions['train']

In [ ]:
emotions_train[0]

{'text': 'i didnt feel humiliated', 'label': 0}

In [ ]:
emotions_train[:3]

{'text': ['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'im grabbing a minute to post i feel greedy wrong'],
 'label': [0, 0, 3]}

In [ ]:
emotions_train.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

In [ ]:
# 원격지에 있는 데이터 파일 로딩: 예 - 네이버 영화 리뷰 감성 분류 코퍼스
base_url = 'https://raw.githubusercontent.com/e9t/nsmc/master/'
data_files = {'train': base_url + 'ratings_train.txt', 'test': base_url + 'ratings_test.txt'}
naver_movie_ds = load_dataset('csv', data_files=data_files, sep='\t')

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
naver_movie_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [ ]:
# 로컬에 저장되어 있는 데이터 파일 로딩 예 (아래 파일이 로컬에 저장되어 있는 경우)
# Colab에서 실행하는 경우 이 파일과 같은 위치에 아래 파일들이 저장되어 있어야 함.
data_files = {'train': local_path+'naver_movie_ratings_train.txt', 'test': local_path+'naver_movie_ratings_test.txt'}
naver_movie_ds = load_dataset('csv', data_files=data_files, sep='\t')

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
naver_movie_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [ ]:
# 물론 파일 하나만 로딩할 수도 있음
# 학습/검증/테스트 분할 정보를 설정하지 않으면, 모두 학습 데이터(train)로 분할됨.
naver_movie_train = load_dataset('csv', data_files=local_path+'naver_movie_ratings_train.txt', sep='\t')
naver_movie_train

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
})

In [ ]:
naver_movie_test = load_dataset('csv', data_files={'test': local_path+'naver_movie_ratings_test.txt'}, sep='\t', split='test')
naver_movie_test

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 50000
})

In [ ]:
naver_movie_train = naver_movie_train['train']
naver_movie_train

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 150000
})

## 토큰분리(Tokenizer)
각 모델에서 사용하는 토큰분리 방식이 조금씩 다르므로, 모델에 적합한 토큰분리 방법을 적용해야 함.

다음 용어를 이해할 필요가 있음.
* 구조(architecture): 모델의 골격 (예: BERT)
* 체크포인트(checkpoint): 주어진 모델 구조에 대한 가중치. 같은 구조이더라도 각종 설정에 따라 서로 다른 가중치로 학습이 이루어질 수 있음. (예: bert-base-uncased, bert-large-cased)
* 모델(model): 구조나 체크포인트를 의미할 수 있는 일반적인 용어.

`transformers.AutoTokenizer`를 사용하면 적합한 토크나이저를 쉽게 로딩할 수 있음.
* `Autotokenizer.from_pretrained('체크포인트 이름')`: 체크포인트 이름에 해당하는 사전학습된 모델과 연관된 토크나이저 로딩. 어휘사전 등.

`AutoTokenizer`의 주요 속성들:
* `vocab_size`: 어휘사전 크기
* `model_max_length`: 최대 문맥 크기
* `model_input_names`: 모델이 순방향 패스(forward pass)에서 기대하는 필드 이름

In [ ]:
from transformers import AutoTokenizer
model_ckpt = 'bert-base-uncased'  # BERT 모델
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
text = 'A tokenizer converts your input into a format that can be processed by the model. 모델에 적합한 토큰분리 방법을 적용해야 함.'
encoded = tokenizer(text)
encoded

{'input_ids': [101, 1037, 19204, 17629, 19884, 2115, 7953, 2046, 1037, 4289, 2008, 2064, 2022, 13995, 2011, 1996, 2944, 1012, 1459, 30011, 29993, 30009, 30022, 29999, 30009, 1464, 30008, 30020, 30005, 30006, 30024, 30005, 30006, 30021, 1467, 30011, 30002, 30017, 30021, 29996, 30014, 30021, 29994, 30019, 1460, 30006, 30025, 29996, 30008, 30024, 29999, 30017, 30022, 100, 1469, 30006, 30023, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

* input_ids: 입력 단어들의 정수 인코딩 결과
* token_type_ids: BERT 입력의 몇번째 문장인지를 나타냄
* attention_mask: 이 값이 0인 입력은 어텐션 계산에 사용되지 않음.

In [ ]:
# 토큰 분리 결과 확인
tokens = tokenizer.convert_ids_to_tokens(encoded.input_ids)
print(tokens)

['[CLS]', 'a', 'token', '##izer', 'converts', 'your', 'input', 'into', 'a', 'format', 'that', 'can', 'be', 'processed', 'by', 'the', 'model', '.', 'ᄆ', '##ᅩ', '##ᄃ', '##ᅦ', '##ᆯ', '##ᄋ', '##ᅦ', 'ᄌ', '##ᅥ', '##ᆨ', '##ᄒ', '##ᅡ', '##ᆸ', '##ᄒ', '##ᅡ', '##ᆫ', 'ᄐ', '##ᅩ', '##ᄏ', '##ᅳ', '##ᆫ', '##ᄇ', '##ᅮ', '##ᆫ', '##ᄅ', '##ᅵ', 'ᄇ', '##ᅡ', '##ᆼ', '##ᄇ', '##ᅥ', '##ᆸ', '##ᄋ', '##ᅳ', '##ᆯ', '[UNK]', 'ᄒ', '##ᅡ', '##ᆷ', '.', '[SEP]']


In [ ]:
# 토큰 시퀀스를 문자열로 변환
sent = tokenizer.convert_tokens_to_string(tokens)
print(sent)

[CLS] a tokenizer converts your input into a format that can be processed by the model. 모델에 적합한 토큰분리 방법을 [UNK] 함. [SEP]


In [ ]:
tokenizer.vocab_size

30522

In [ ]:
tokenizer.model_max_length

512

In [ ]:
tokenizer.model_input_names

['input_ids', 'token_type_ids', 'attention_mask']

## 모델 로딩
`transformers.AutoTokenizer`로 적합한 토크나이저를 로딩했듯이, `transformers.AutoModel`을 이용하여 적합한 모델을 로딩할 수 있음. 허깅페이스 트랜스포머스는 기본적으로 PyTorch에 최적화되어 있지만, Tensorflow와의 상호운영성을 제공함. Tensorflow에 대응되는 클래스는 앞에 'TF'만 붙이면 됨. 즉, `AutoModel` 클래스에 대응되는 Tensorflow용 클래스는 `TFAutoModel` 클래스임.

* `AutoModel.from_pretrained('체크포인트 이름')`: 체크포인트 이름에 해당하는 사전학습된 모델 로딩. 모델 설정, 사전학습된 가중치 등(PyTorch용). 텐서플로용으로만 릴리즈된 모델인 경우, `from_tf=True`로 파라미터를 추가 설정하여 파이토치용으로 변환할 수 있음.
* `TFAutoModel.from_pretrained('체크포인트 이름')`: 텐서플로용. 만일 파이토치용으로만 릴리즈된 모델인 경우, `from_pt=True`로 설정하여 텐서플로용으로 변환할 수 있음.

In [ ]:
import torch
from transformers import AutoModel

model_ckpt = 'bert-base-uncased'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPU or CPU
print(f'Using {device} device...')
model = AutoModel.from_pretrained(model_ckpt).to(device)

Using cuda device...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# 토큰 분리 결과를 모델의 입력으로 이용하려면, 토크나이징 결과를 텐서 형식으로 리턴받아야 함.
# return_tensors 파라미터를 설정해주면 됨. 파이토치용은 'pt', 텐서플로용은 'tf'.
inputs = tokenizer(text, return_tensors='pt')
inputs

{'input_ids': tensor([[  101,  1037, 19204, 17629, 19884,  2115,  7953,  2046,  1037,  4289,
          2008,  2064,  2022, 13995,  2011,  1996,  2944,  1012,  1459, 30011,
         29993, 30009, 30022, 29999, 30009,  1464, 30008, 30020, 30005, 30006,
         30024, 30005, 30006, 30021,  1467, 30011, 30002, 30017, 30021, 29996,
         30014, 30021, 29994, 30019,  1460, 30006, 30025, 29996, 30008, 30024,
         29999, 30017, 30022,   100,  1469, 30006, 30023,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
inputs['input_ids'].size()  # [배치크기, 토큰개수]

torch.Size([1, 59])

In [ ]:
for k, v in inputs.items():
    print(k, v, sep='\t')

input_ids	tensor([[  101,  1037, 19204, 17629, 19884,  2115,  7953,  2046,  1037,  4289,
          2008,  2064,  2022, 13995,  2011,  1996,  2944,  1012,  1459, 30011,
         29993, 30009, 30022, 29999, 30009,  1464, 30008, 30020, 30005, 30006,
         30024, 30005, 30006, 30021,  1467, 30011, 30002, 30017, 30021, 29996,
         30014, 30021, 29994, 30019,  1460, 30006, 30025, 29996, 30008, 30024,
         29999, 30017, 30022,   100,  1469, 30006, 30023,  1012,   102]])
token_type_ids	tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask	tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [ ]:
# 입력 텐서들을 모델이 있는 디바이스로 이동하여 모델의 입력으로 전달
inputs = {k:v.to(device) for k,v in inputs.items()}
with torch.no_grad(): # 추론에 이용할 것이므로, 그레이디언트 자동 계산 비활성화
    outputs = model(**inputs)
print(outputs)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3712, -0.4148, -0.1034,  ..., -0.1212,  0.3965,  0.2409],
         [-0.7397, -0.3869, -0.0345,  ..., -0.2732, -0.2331,  0.4034],
         [-0.6785, -0.1093, -0.7495,  ..., -0.1165,  0.1407,  0.1638],
         ...,
         [-0.7598, -1.1164,  0.2230,  ...,  0.1094, -0.0687, -1.0860],
         [-0.9193, -1.1609, -0.7973,  ...,  0.6783,  0.2631, -0.7882],
         [ 0.1261, -0.1069, -0.4341,  ...,  0.0082, -0.1659, -0.1079]]],
       device='cuda:0'), pooler_output=tensor([[-0.8633, -0.3548, -0.9780,  0.8630,  0.8620, -0.3442,  0.6805,  0.2615,
         -0.9182, -1.0000, -0.7005,  0.8768,  0.9132,  0.7897,  0.7590, -0.7818,
         -0.3491, -0.6777,  0.4078, -0.0032,  0.5929,  1.0000, -0.4408,  0.3881,
          0.4890,  0.9835, -0.7026,  0.7900,  0.8442,  0.5371, -0.6568,  0.2641,
         -0.9556, -0.4048, -0.9892, -0.9789,  0.5736, -0.5152, -0.2064, -0.0567,
         -0.8421,  0.3780,  1.0000,  0.1422,  0.647

In [ ]:
# 마지막 층의 은닉 상태가 중요함
outputs.last_hidden_state

tensor([[[-0.3712, -0.4148, -0.1034,  ..., -0.1212,  0.3965,  0.2409],
         [-0.7397, -0.3869, -0.0345,  ..., -0.2732, -0.2331,  0.4034],
         [-0.6785, -0.1093, -0.7495,  ..., -0.1165,  0.1407,  0.1638],
         ...,
         [-0.7598, -1.1164,  0.2230,  ...,  0.1094, -0.0687, -1.0860],
         [-0.9193, -1.1609, -0.7973,  ...,  0.6783,  0.2631, -0.7882],
         [ 0.1261, -0.1069, -0.4341,  ...,  0.0082, -0.1659, -0.1079]]],
       device='cuda:0')

In [ ]:
outputs.last_hidden_state.size()  # [배치크기, 토큰개수, 은닉층 차원]

torch.Size([1, 59, 768])

In [ ]:
# 분류 작업 시에는 첫번째 토큰인 [CLS] 토큰의 은닉 상태를 이용함
outputs.last_hidden_state[:, 0]

tensor([[-3.7115e-01, -4.1479e-01, -1.0345e-01, -4.5756e-01, -7.9443e-01,
         -8.3000e-02,  3.6424e-01,  4.4675e-01,  4.9328e-02, -6.1216e-01,
          6.8209e-02, -1.5920e-01, -4.9029e-01, -2.6235e-01,  9.6650e-02,
          3.8486e-01,  1.1660e-01,  7.0872e-01,  2.9854e-01, -4.8420e-02,
         -5.6197e-01, -5.0343e-01,  3.8446e-02, -1.3203e-01,  3.0661e-01,
         -5.5026e-01, -6.8818e-02, -3.6687e-02,  2.5194e-01, -6.8638e-02,
         -2.9257e-02,  2.2429e-01, -3.1114e-01, -7.7947e-01,  9.0415e-01,
          3.8458e-02,  1.0734e-01, -3.3281e-01,  1.9760e-01,  1.1457e-01,
         -4.2469e-01,  4.9454e-02,  2.9068e-01, -3.0476e-01,  2.3759e-01,
         -5.3377e-01, -3.6157e+00,  3.4841e-01, -1.0369e+00, -7.8937e-01,
          1.4080e-01,  1.1394e-01, -4.0696e-01,  4.7679e-01, -9.2246e-02,
          4.2377e-01, -3.0782e-01,  1.2564e-01,  1.9167e-01,  3.1400e-01,
          1.1678e-01,  1.2746e-01,  5.8679e-03, -1.8751e-01, -2.2236e-01,
          1.9841e-01, -5.8379e-01,  4.

## 활용 예: Transformers를 이용한 텍스트 분류
한국어 처리를 위해 `klue/bert-base` 모델 사용

### 데이터셋 준비
네이버 영화 리뷰 감성 분류

In [ ]:
# 네이버 영화 리뷰 감성 분류 코퍼스 로딩
from datasets import load_dataset

base_url = 'https://raw.githubusercontent.com/e9t/nsmc/master/'
data_files = {'train': base_url + 'ratings_train.txt', 'test': base_url + 'ratings_test.txt'}
naver_movie_ds = load_dataset('csv', data_files=data_files, sep='\t')
naver_movie_ds

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [ ]:
# 데이터 전처리 등을 수행하기 위해서는 판다스의 데이터프레임 기능이 유용함.
# Dataset 객체를 판다스 데이터프레임 포맷으로 변환이 가능함.
naver_movie_ds.set_format(type='pandas')
naver_movie_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [ ]:
naver_movie_ds['train'].format

{'type': 'pandas',
 'format_kwargs': {},
 'columns': ['id', 'document', 'label'],
 'output_all_columns': False}

In [ ]:
train_df = naver_movie_ds['train'][:]
test_df = naver_movie_ds['test'][:]

In [ ]:
type(train_df)

pandas.core.frame.DataFrame

In [ ]:
type(train_df)

pandas.core.frame.DataFrame

In [ ]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
# 학습 데이터: 중복제거, 널 값 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df = train_df.dropna(how='any')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146182 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        146182 non-null  int64 
 1   document  146182 non-null  object
 2   label     146182 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.5+ MB


In [ ]:
# 테스트 데이터: 중복제거, 널 값 제거
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df = test_df.dropna(how='any')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49157 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49157 non-null  int64 
 1   document  49157 non-null  object
 2   label     49157 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [ ]:
# 전처리 완료된 데이터프레임으로부터 데이터셋 생성
from datasets import Dataset, DatasetDict
naver_movie_ds = DatasetDict({'train': Dataset.from_pandas(train_df), 'test': Dataset.from_pandas(test_df)})

In [ ]:
naver_movie_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label', '__index_level_0__'],
        num_rows: 146182
    })
    test: Dataset({
        features: ['id', 'document', 'label', '__index_level_0__'],
        num_rows: 49157
    })
})

In [ ]:
naver_movie_ds['train'][:3]

{'id': [9976970, 3819312, 10265843],
 'document': ['아 더빙.. 진짜 짜증나네요 목소리',
  '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
  '너무재밓었다그래서보는것을추천한다'],
 'label': [0, 1, 0],
 '__index_level_0__': [0, 1, 2]}

### 토큰 분리

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

model_ckpt = 'klue/bert-base'

# 토큰 분리를 위한 토크나이저 로딩
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
# 토큰 분리를 위한 함수 정의
def tokenize(batch):
    return tokenizer(batch['document'], padding=True, truncation=True)

In [ ]:
# 데이터셋 전체에 대해 토큰 분리 적용: 위에서 정의한 tokenize 함수 적용
# 데이터셋의 map() 함수 이용
# 별도 설정하지 않으면 각 샘플을 개별적으로 적용함
# 여러 샘플에 한꺼번에 적용하려면, batched, batch_size(기본값 1000) 파라미터를 설정
# batch_size가 0 이하이거나 None이면 모든 샘플을 하나의 배치로 처리함.
# 하나의 배치로 전체를 처리해야 입력 텐서와 어텐션 마스크가 전체적으로 동일한 크기로 생성됨
encoded = naver_movie_ds.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/146182 [00:00<?, ? examples/s]

Map:   0%|          | 0/49157 [00:00<?, ? examples/s]

In [ ]:
print(encoded['train'][:3])

{'id': [9976970, 3819312, 10265843], 'document': ['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다'], 'label': [0, 1, 0], '__index_level_0__': [0, 1, 2], 'input_ids': [[2, 1376, 831, 2604, 18, 18, 4229, 9801, 2075, 2203, 2182, 4243, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 1963, 18, 18, 18, 11811, 2178, 2088, 28883, 16516, 2776, 18, 18, 18, 18, 10737, 2156, 2015, 2446, 2232, 6758, 2118, 1380, 6074, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
encoded['train'].column_names

['id',
 'document',
 'label',
 '__index_level_0__',
 'input_ids',
 'token_type_ids',
 'attention_mask']

In [ ]:
print(tokenizer.convert_ids_to_tokens(encoded['train'][0]['input_ids']))

['[CLS]', '아', '더', '##빙', '.', '.', '진짜', '짜증', '##나', '##네', '##요', '목소리', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

### 트랜스포머 파인튜닝하기
특정 태스크 수행을 위한 주요 파인튜닝 모델들은 다음과 같음. 사전 학습된 언어모델(베이스 모델) 위에 특정 태스크 수행에 적합한 형태의 레이어(head)가 추가된 구조임. 각 모델의 `from_pretrained(체크포인트명)`을 통해 사전 학습된 언어모델을 로딩함. 이후 해당 태스크용 데이터셋을 사용하여 추가 학습(파인튜닝)해야 함.

* `AutoModelForMaskedLM`: 마스크 언어 모델링
* `AutoModelForNextSentencePrediction`: 다음 문장 예측.
* `AutoModelForSequenceClassification`: 시퀀스 분류. 예: 텍스트 분류
* `AutoModelForTokenClassification`: 토큰 분류. 각 토큰에 레이블 지정. 예: 개체명 인식, 품사 태깅
* `AutoModelForSeq2SeqLM`: sequence-to-sequence 언어 모델링. 예: 번역, 요약, 생성
* `AutoModelForMultipleChoice`: 여러 보기 중에서 선택하는 문제.
* `AutoModelForQuestionAnswering`: (추출형) 질의응답. 예: 기계 독해. (질문, 문맥) -> 답변

상세 사항은 여기 참고: https://huggingface.co/docs/transformers/model_doc/auto#natural-language-processing

`Trainer`를 통해서 학습을 진행하고 학습에 사용할 하이퍼 파라미터들을 `TrainingArguments`로 정의함.

`TrainingArguments`의 주요 파라미터들:
* output_dir: 모델 체크포인트와 예측들이 저장될 디렉토리
* num_train_epochs: 학습 에포크 수. 기본값 3.0
* learning_rate: AdamW 옵티마이저의 학습율 초기값. 기본값 5e-5
* per_device_train_batch_size: 학습 시 GPU/TPU/CPU 하나 당 배치 크기. 기본값 8
* per_device_eval_batch_size: 평가 시 GPU/TPU/CPU 하나 당 배치 크기. 기본값 8
* weight_decay: 과적합 방지를 위해 AdamW 옵티마이저에서 편향(bias)과 LayerNorm 가중치를 제외한 모든 층에 적용할 가중치 감쇠값. 0~1. 높을수록 더 강한 가중치 감쇠가 적용됨. 0이면 적용되지 않음. 기본값 0.
* evaluation_strategy: 학습 중 평가 방법. 기본값 'no'
  * 'no': 평가 미실시
  * 'steps': `eval_steps`마다 평가
  * 'epoch': 각 에포크 완료 시마다 평가
* disable_tqdm: 진행 상태 바와 성능 수치 표를 보이지 않도록 비활성화할 것인지의 여부. 기본값 True
* logging_strategy: 학습 중 로깅 방법. 기본값 'steps'
  * 'no': 로깅하지 않음
  * 'steps': `logging_steps`마다
  * 'epoch': 각 에포크 완료 시마다
* logging_steps: 두 로그 사이의 간격(업데이트 스텝의 수)
* push_to_hub: 모델이 저장될 때마다 모델을 허깅페이스 허브에 푸시할 것인지의 여부. 기본값 False
* save_strategy: 학습 중 체크포인트 저장 방법. 기본값 'steps'
  * 'no': 저장하지 않음
  * 'steps': `save_steps`마다
  * 'epoch': 각 에포크 완료 시마다
* load_best_model_at_end: 학습 종료 시 학습 중 발견된 베스트 모델을 로딩할 것인지의 여부. 기본값 False
* log_level: 로그 레벨. 기본값 'passive'

상세 사항은 여기 참고: https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments

`Trainer`의 주요 파라미터들:
* model: 학습할 모델
* args: 학습에 사용할 하이퍼 파라미터들(`TrainingArguments`)
* compute_metrics: 평가 시 계산할 성능 척도를 정의하는 함수. `EvalPrediction`을 입력 받아, {'척도명': 값} 형식의 딕셔너리를 리턴
  * `EvalPrediction`의 주요 속성:
    * label_ids: 정답 레이블(np.ndarray)
    * predictions: 모델의 예측 결과(np.ndarray)
* train_dataset: 학습 데이터
* eval_dataset: 학습 중 평가에 사용되는 검증 데이터
* tokenizer: 데이터 전처리에 사용할 토크나이저

`Trainer.train()` 함수를 통해 학습 수행. 학습 결과 모델을 이용하여 예측하려면 `Trainer.predict(데이터셋)` 함수 사용. `predict()` 함수는 다음의 값을 담은 `NamedTuple`을 리턴함
* predictions: 데이터셋에 대한 예측 결과(np.ndarray)
* label_ids: 데이터셋에 포함되어 있었던 경우에만, 정답 레이블(np.ndarray)
* metrics: 데이터셋에 포함되어 있었던 경우에만, {'척도명': 값} 형식의 성능 평가 결과 딕셔너리

상세 사항은 여기 참고: https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer

In [ ]:
# 베이스 모델 + 분류기
from transformers import AutoModelForSequenceClassification

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPU or CPU
print(f'Using {device} device...')

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device)

Using cuda device...


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [ ]:
# 성능 척도 정의
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
# 학습 파라미터 정의
# 학습 도중 OutOfMemoryError 발생 시, batch_size를 적절히 줄인 후
# Jupyter Notebook Kernel을 재시작한 후 다시 시도하면 됨.
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(encoded["train"]) // batch_size
model_name = local_path + f"{model_ckpt}-finetuned-nmovie"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False,
                                  save_strategy="epoch",
                                  load_best_model_at_end=True,
                                  log_level="error")

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=encoded["train"],
                  eval_dataset=encoded["test"],
                  tokenizer=tokenizer)
trainer.train();

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.273700,0.241223,0.902130,0.902096
2,0.185100,0.241618,0.906361,0.906358


In [ ]:
pred_output = trainer.predict(encoded["test"])
pred_output.metrics

{'test_loss': 0.24122266471385956,
 'test_accuracy': 0.9021299102874464,
 'test_f1': 0.9020960632927343,
 'test_runtime': 374.006,
 'test_samples_per_second': 131.434,
 'test_steps_per_second': 2.056}

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

y_valid = np.array(encoded["test"]["label"])
y_pred = np.argmax(pred_output.predictions, axis=1)
confusion_matrix(y_valid, y_pred)

array([[21645,  2801],
       [ 2010, 22701]])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.89      0.90     24446
           1       0.89      0.92      0.90     24711

    accuracy                           0.90     49157
   macro avg       0.90      0.90      0.90     49157
weighted avg       0.90      0.90      0.90     49157

